In [3]:
import xarray as xr 
import numpy as np
import os 
from glob import glob
import tensorflow as tf 

In [5]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

2022-11-11 11:37:37.970053: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38247 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0


In [9]:
date = '20210504'
init_time = '2300'

In [30]:
BASE_PATH = '/work/brian.matilla/WOFS_2021/summary_files/WOFS_JET/WOFS_1KM/'
train_paths = glob(os.path.join(BASE_PATH, date, init_time, f'wofs_ENS_*'))

def resize(img, height, width):
    img = tf.image.resize(img, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return img

def load(file_path):
    N_ENS_MEM = 18 
    N_PATCHES = 7*7
    
    MEAN = 23.30260807740146
    STD = 16.574040989743658
    
    ds = xr.load_dataset(file_path, decode_times=False)
    dbz = ds['comp_dz'].values[:,:,:]
    
    # Normalize the data. 
    dbz = (dbz - MEAN) / STD 
    
    # Reshape (N, 402, 402, 1)
    dbz = dbz.reshape(list(dbz.shape)+[1])
    
    
    # Convert image to float32 
    dbz = tf.cast(dbz, tf.float32)
    
    # Create patches 
    dbz_patches = tf.image.extract_patches(dbz, 
                                           sizes=[1, 64, 64,1], 
                                           strides=[1,64, 64, 1],
                                           rates=[1,1,1,1],
                                           padding='SAME')

    # Reshape
    dbz_patches = tf.reshape(dbz_patches, (N_PATCHES*N_ENS_MEM, 64, 64, 1))
    
    return dbz_patches

In [31]:
ds = load(train_paths[0])

23.30260807740146 16.574040989743658


In [32]:
ds.shape

TensorShape([882, 64, 64, 1])